#Kirsten Mayland - Final Project

---

Kirsten Mayland (kirsten.r.mayland.25@dartmouth.edu) <br>
Dartmouth College, CS72, Winter 2025

Purpose: To create a database from Reddit's r/AskDocs subreddit that has three columns [post_title, post_text, comment_questions] in order to train an LLM on it

# Current Attempt (Using downloaded data)

---



## Set Up

---



In [15]:
import zstandard
import os
import json
import sys
import csv
from datetime import datetime
import logging.handlers
import asyncio

## Downloading Initial Data

---



Downloaded all the posts and comments in r/AskDocs from May, 2005 to December, 2024 using u/postshift's database.

Intial post: <br>
https://www.reddit.com/r/pushshift/comments/1itme1k/separate_dump_files_for_the_top_40k_subreddits/

Website to download: <br>
https://academictorrents.com/details/1614740ac8c94505e4ecb9d88be8bed7b6afddd4

## Converting Initial Zip Files to Wanted CSV files

---



The database contained all the data for each post and comment in two seperate files:
* "AskDocs_comments.zst"
* "AskDocs_submissions.zst"

This section takes those two zip files and creates two csv files that only contains the necessary information: <br>
* "AskDocs_comments.csv"
*  "AskDocs_submissions.csv"

For "AskDocs_comments.csv" that is ["author","author_flair_text","body","link_id"] and for "AskDocs_submissions.csv" that is ["name", "title", "selftext", "num_comments","url"]. <br>

<br>

Most of this code was modified from:  https://github.com/Watchful1/PushshiftDumps/blob/master/scripts/to_csv.py

In [18]:
from google.colab import files
# (Upload using sidebar now bc faster)
# uploaded = files.upload()

In [ ]:
# Imported from: https://github.com/Watchful1/PushshiftDumps/blob/master/scripts/to_csv.py
def read_and_decode(reader, chunk_size, max_window_size, previous_chunk=None, bytes_read=0):
	chunk = reader.read(chunk_size)
	bytes_read += chunk_size
	if previous_chunk is not None:
		chunk = previous_chunk + chunk
	try:
		return chunk.decode()
	except UnicodeDecodeError:
		if bytes_read > max_window_size:
			raise UnicodeError(f"Unable to decode frame after reading {bytes_read:,} bytes")
		return read_and_decode(reader, chunk_size, max_window_size, chunk, bytes_read)

# Imported from: https://github.com/Watchful1/PushshiftDumps/blob/master/scripts/to_csv.py
def read_lines_zst(file_name):
	with open(file_name, 'rb') as file_handle:
		buffer = ''
		reader = zstandard.ZstdDecompressor(max_window_size=2**31).stream_reader(file_handle)
		while True:
			chunk = read_and_decode(reader, 2**27, (2**29) * 2)
			if not chunk:
				break
			lines = (buffer + chunk).split("\n")

			for line in lines[:-1]:
				yield line, file_handle.tell()

			buffer = lines[-1]
		reader.close()

In [ ]:
# Imported and modified from: https://github.com/Watchful1/PushshiftDumps/blob/master/scripts/to_csv.py
# Take zip file and create a csv file with only the information we need
async def run_cull(input_file_path, output_file_path):
  is_submission = "submission" in input_file_path
  if is_submission:
    print("Is submission")
    fields = ["name", "title", "selftext", "num_comments","url"]
  else:
    print("Is comment")
    fields = ["author","author_flair_text","body","link_id"]

  file_size = os.stat(input_file_path).st_size
  file_lines, bad_lines = 0, 0
  line, created = None, None
  output_file = open(output_file_path, "w", encoding='utf-8-sig', newline="")
  writer = csv.writer(output_file)
  writer.writerow(fields)
  try:
    for line, file_bytes_processed in read_lines_zst(input_file_path):
      try:
        has_broken = False

        obj = json.loads(line)
        output_obj = []

        # go through the fields we want to keep, and either store the data or discard the line
        for field in fields:
          if field == "author":
            if 'author' in obj:
              # if author is AutoModerator discard comment/post
              if obj['author'] == "AutoModerator":
                has_broken = True
                break
              value = f"u/{obj['author']}"
            else:
              has_broken = True
              break

          elif field == "author_flair_text":
            if 'author_flair_text' in obj:
              value = obj['author_flair_text']
              if value is None:
                has_broken = True
                break
              # don't accept comments from non-medical professionals
              value_lower = value.lower()
              if "layperson" in value_lower:
                has_broken = True
                break
              if "not yet been verified" in value_lower:
                has_broken = True
                break
            else:
              has_broken = True
              break

          elif field == "selftext":
            if 'selftext' in obj:
              value = obj['selftext']
            else:
              has_broken = True
              break

          elif field == "body":
            # only keep comment if has a question
            if 'body' in obj:
              value = obj['body']
              if "?" not in value:
                has_broken = True
                break
            else:
              has_broken = True
              break

          elif field == "num_comments":
            value = obj['num_comments']
            if value is None or value == 0:
              has_broken = True
              break

          elif field == "title":
            value = obj['title']

          elif field == "name":
            if 'name' in obj:
              value = obj['name']
            else:
              has_broken = True
              break

          # removes the nested comments
          elif field == "link_id":
            if 'link_id' in obj:
              value = obj['link_id']
              if not value.startswith("t3_"):
                has_broken = True
                break
            else:
              has_broken = True
              break

          else:
            value = obj[field]

          # break out of nested loop, no need to look at more fields
          if has_broken:
            continue

          output_obj.append(str(value).encode("utf-8-sig", errors='replace').decode())

        # if something went wrong, don't write to output
        if not has_broken:
          writer.writerow(output_obj)

        created = datetime.utcfromtimestamp(int(obj['created_utc']))
      except json.JSONDecodeError as err:
        bad_lines += 1
      file_lines += 1
      if file_lines % 100000 == 0:
        log.info(f"{created.strftime('%Y-%m-%d %H:%M:%S')} : {file_lines:,} : {bad_lines:,} : {(file_bytes_processed / file_size) * 100:.0f}%")
  except KeyError as err:
    log.info(f"Object has no key: {err}")
    log.info(line)
  except Exception as err:
    log.info(err)
    log.info(line)

  output_file.close()
  files.download(output_file_path)
  log.info(f"Complete : {file_lines:,} : {bad_lines:,}")

In [ ]:
log = logging.getLogger("bot")
log.setLevel(logging.DEBUG)
log.addHandler(logging.StreamHandler())

# Note, due to the way files.download is set up and the openning and closing of files,
# please only run one at a time and comment out the other
async def main():
  await run_cull("/content/sample_data/AskDocs_comments.zst", "AskDocs_comments.csv")
  # await run_cull("/content/sample_data/AskDocs_submissions.zst", "AskDocs_submissions.csv")

await main()

Is comment


2015-07-30 17:29:24 : 100,000 : 0 : 5%
2015-07-30 17:29:24 : 100,000 : 0 : 5%
INFO:bot:2015-07-30 17:29:24 : 100,000 : 0 : 5%
2016-06-15 17:35:25 : 200,000 : 0 : 9%
2016-06-15 17:35:25 : 200,000 : 0 : 9%
INFO:bot:2016-06-15 17:35:25 : 200,000 : 0 : 9%
2017-02-09 16:48:10 : 300,000 : 0 : 9%
2017-02-09 16:48:10 : 300,000 : 0 : 9%
INFO:bot:2017-02-09 16:48:10 : 300,000 : 0 : 9%
2017-09-03 16:46:58 : 400,000 : 0 : 13%
2017-09-03 16:46:58 : 400,000 : 0 : 13%
INFO:bot:2017-09-03 16:46:58 : 400,000 : 0 : 13%
2018-02-14 19:05:45 : 500,000 : 0 : 13%
2018-02-14 19:05:45 : 500,000 : 0 : 13%
INFO:bot:2018-02-14 19:05:45 : 500,000 : 0 : 13%
2018-07-04 23:49:52 : 600,000 : 0 : 16%
2018-07-04 23:49:52 : 600,000 : 0 : 16%
INFO:bot:2018-07-04 23:49:52 : 600,000 : 0 : 16%
2018-11-08 22:21:21 : 700,000 : 0 : 18%
2018-11-08 22:21:21 : 700,000 : 0 : 18%
INFO:bot:2018-11-08 22:21:21 : 700,000 : 0 : 18%
2019-02-10 05:21:33 : 800,000 : 0 : 19%
2019-02-10 05:21:33 : 800,000 : 0 : 19%
INFO:bot:2019-02-10 05:21:

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Complete : 4,966,225 : 0
Complete : 4,966,225 : 0
INFO:bot:Complete : 4,966,225 : 0


## Combine, Normalize, and Process Two CSV Files into One

---



In [7]:
import pandas as pd
import numpy as np
import html
import re

In [8]:
def clean_text(text):
  # remove byte order marks
  text = text.replace("\ufeff", "")

  # replace newline characters with a space
  text = text.replace("\n", " ")

  # unescape html
  text = html.unescape(text)

  return text

In [9]:
def create_merged_data_array():
  # load the CSV files into pandas DataFrames
  submissions_df = pd.read_csv("AskDocs_submissions.csv")
  comments_df = pd.read_csv("AskDocs_comments.csv")

  # remove byte order marks
  submissions_df["title"] = submissions_df["title"].apply(clean_text)
  submissions_df["selftext"] = submissions_df["selftext"].apply(clean_text)
  comments_df["body"] = comments_df["body"].apply(clean_text)

  # clean the submissions df by replacing NaN with empty strings
  # and removing rows where selftext is missing or is "[removed]" or is "[deleted]".
  submissions_df["selftext"] = submissions_df["selftext"].fillna("").replace({"[removed]": "", "[deleted]": ""}).str.strip()
  submissions_df = submissions_df[submissions_df["selftext"] != ""]

  # clean the comments df by replacing NaN with empty strings
  # and removeing comments that are empty or "[removed]" "[deleted]"
  comments_df["body"] = comments_df["body"].fillna("").replace({"[removed]": "", "[deleted]": ""}).str.strip()
  comments_df = comments_df[comments_df["body"] != ""]

  # merge the submissions and comments on submission identifier
  # 'name' in submissions == 'link_id' in comments
  merged_df = submissions_df.merge(comments_df, left_on="name", right_on="link_id", how="inner")

  # group by submission fields to aggregate all comment bodies into a list
  grouped = merged_df.groupby(["name", "title", "selftext"])["body"].apply(list).reset_index()

  # only keep submissions with at least one comment (non-empty list)
  grouped = grouped[grouped["body"].apply(lambda x: len(x) > 0)]

  # create the array in the format: [title, selftext, comment1, comment2, ...]
  merged_array = grouped.apply(lambda row: [row["title"], row["selftext"]] + row["body"], axis=1).tolist()


  print(f"Length of final array = {len(merged_array)}")
  for row in merged_array[:5]:
      print(row)

  return merged_array

## Condense All Comments into Question List


---



In [10]:
# go through each post with relevant comments and get a formatted list of the follow-up questions
def extract_questions(final_array_row):
  removed_questions = []
  filtered_questions = []
  for index in range(2, len(final_array_row)):
    comment_body = final_array_row[index]

    # if starts at the beginning of text/period/exclamation mark/question mark and ends with a question mark
    # (not allowed to start with or contain > bc that would be a reference)
    question_matches = re.findall(r'(?:^|\n|\.|!|\?|\()(\([^\n.!?>]*\)\?|[^\n.!?>]*\?)', comment_body.strip())
    found = [question.strip() for question in question_matches]

    # list of phrases that indicate a recommendation, with allowance for an adjective in between
    exclude_patterns = [r"i\s+(?:\w+\s+)?recommend", r"i\s+(?:\w+\s+)?suggest", r"i\s+(?:\w+\s+)?suspect"]
    # list of common unhelpful one word questions generated by ChatGPT
    exclude_one_word_patterns = [r"okay", r"ok", r"why", r"how", r"what", r"who",
                                 r"where", r"which", r"sure", r"really", r"explain",
                                 r"someone", r"anything", r"anyone", r"jpeg",
                                 r"maybe"]
    exclude_link_patterns = [
        r"https?://\S+",   # Full URLs (http:// or https://)
        r"www\.\S+",       # URLs starting with www.
        r"\b(?:[a-zA-Z0-9.-]+\.)?(com|org|net|gov|edu|io|co)(/\S*)?\b"  # Detects domain-like paths
    ]

    for q in found:
      # remove recommendations
      if any(re.search(pattern, q.lower()) for pattern in exclude_patterns):
        removed_questions.append(q)
      # remove unhelpful one word questions like "why?"
      elif len(q.split()) == 1 and any(re.search(pattern, q.lower()) for pattern in exclude_one_word_patterns):
        removed_questions.append(q)
      # remove links
      elif any(re.search(pattern, q.lower()) for pattern in exclude_link_patterns):
        removed_questions.append(q)
      # has at least one letter
      elif re.search(r"[a-zA-Z]", q) is None:
        removed_questions.append(q)
      else:
        filtered_questions.append(q)

  # print if removed questions
  if removed_questions:
    print(f"removed_questions = {removed_questions}")

  if not filtered_questions:
    return None

  return filtered_questions

In [11]:
def format_question_list(question_list):
    return "\n".join([f"{i+1}) {q}" for i, q in enumerate(question_list)])

In [12]:
merged_array = create_merged_data_array()

# extracts actual questions, i.e. not references to questions the poster asked
final_post_questions_array = []

for row in merged_array:
  # extract questions from all comments in the row
  question_list = extract_questions(row)

  # only add the row if there is at least one extracted question
  if question_list is not None:
    formatted_questions = format_question_list(question_list)
    final_post_questions_array.append([row[0], row[1], formatted_questions])

Length of final array = 58655
['Do my (35M) stress echo results mean I have had a prior heart attack', '35 years old, white male, 210lbs, 5’9”, 32bmi   Former smoker - quit 8+ years ago   Been experience fatigue, headaches, lightheaded, facial flushing, rapid heart beat, High blood pressure   I had an echo which showed several abnormalities and doctor requested stress echo. I saw the below in the report and don’t speak with him until Tuesday due to the a holiday. Does this mean I likely had a heart attack at some point and didn’t know it?   ECG   NSR, normal axis, HR 93 bpm, Q waves in III, AVF suggestive for old inferior MI. Poor R wave progression  Indications: 10 - Cardiomyopathy, Unspecified - I42.9.', 'Do you mean a stress echo or a stress EKG? The results you show are an EKG. It’s hard to comment without seeing the tracings, and what I think the results are describing are somewhat nonspecific. Your cardiologist, whom will review the actual data, will be able to answer your questi

In [13]:
# print to check
print(f"Length of final array = {len(final_post_questions_array)}")
for item in final_post_questions_array[:10]:
  print(f"Title = {item[0]}")
  print(f"Post = {item[1]}")
  print(f"Questions = {item[2]}")
  print("")

Length of final array = 55887
Title = Do my (35M) stress echo results mean I have had a prior heart attack
Post = 35 years old, white male, 210lbs, 5’9”, 32bmi   Former smoker - quit 8+ years ago   Been experience fatigue, headaches, lightheaded, facial flushing, rapid heart beat, High blood pressure   I had an echo which showed several abnormalities and doctor requested stress echo. I saw the below in the report and don’t speak with him until Tuesday due to the a holiday. Does this mean I likely had a heart attack at some point and didn’t know it?   ECG   NSR, normal axis, HR 93 bpm, Q waves in III, AVF suggestive for old inferior MI. Poor R wave progression  Indications: 10 - Cardiomyopathy, Unspecified - I42.9.
Questions = 1) Do you mean a stress echo or a stress EKG?

Title = Bumpy red rash on the backs of both hands.
Post = 30f, taking multivitamin, currently on day 4 of covid. I will post pictures in the comments!   I've gotten these sorts of small red fluid filled bumps on my ha

## Export Final Array Data to CSV


---



In [19]:
csv_file = "CS72_FinalProject_Reddit_AskDocs_Dataset.csv"

# open a CSV file for writing
with open(csv_file, mode="w", encoding='utf-8-sig', newline="") as file:
  writer = csv.writer(file)

  # write the header (first row)
  writer.writerow(["Title", "Post", "Follow-Up Questions"])

  # write each post and its associated questions
  for row in final_post_questions_array[0:]:
    # convert question list to formatted string
    writer.writerow([row[0], row[1], row[2]])

print(f"Data exported to {csv_file}")
file.close()
files.download(csv_file)

Data exported to CS72_FinalProject_Reddit_AskDocs_Dataset.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Old Attempt (Using Reddit API) (Ignore)

---

<br>

Because of Reddit's 1000 post rate limit, I was only able to get 62 data points max and thus had to abandon this attempt. I'm leaving it here commented out for future reference

## Set Up


---



In [ ]:
# !pip install asyncpraw

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# import asyncpraw
# import asyncio
# import csv
# import re

## Gather Data from r/AskDocs

---



In [ ]:
# # use global to save data if it crashes
# original_posts_data = []
# filtered_posts_data = []
# last_post = None
# i = 0

In [ ]:
# # loops through reddit posts and
# # compiles a list of data lists for each post with relevant comments
# async def fetch_posts(reddit, number_of_posts_to_find):
#   global original_posts_data
#   global filtered_posts_data
#   global last_post
#   global i

#   subreddit = await reddit.subreddit("AskDocs")
#   print(f"Accessed subreddit.")

#   # loop through posts
#   params = {"after": last_post} if last_post else {}
#   async for post in subreddit.new(limit=100, params=params):
#     i += 1
#     print(f"Accessing post {i}.")
#     await post.load()

#     # to stay within free reddit api rate limits
#     await asyncio.sleep(2)
#     if i % 200 == 0:
#       print("Pausing for 30 seconds to avoid rate limits...")
#       await asyncio.sleep(30)

#     # if there is comments, evalute the post
#     if post.comments:
#       await post.comments.replace_more(limit=None)
#       comment_data = []
#       all_questions = []

#       # for each comment under the post, check to see if it fits our needs
#       for comment in post.comments.list():
#         flair_text = comment.author_flair_text or "Unknown"
#         is_not_med_prof = await is_not_medical_professional(comment.author, flair_text)

#         if flair_text and not is_not_med_prof:
#           # if by medical professional, check if has a question in it
#           if '?' in comment.body:
#             # process and extract the relevant questions
#             questions = extract_questions(comment.body)
#             if questions:
#               all_questions.extend(questions)

#             comment_data.append((comment.body, flair_text))

#       # if the post had questions, add it to the list
#       if comment_data:
#         print(f"Post has questions in comments.")
#         # store the dataset without filtered questions
#         row_original = [post.url, post.title, post.selftext, all_questions if all_questions else "No relevant questions"] + [item for sublist in comment_data for item in sublist]
#         original_posts_data.append(row_original)

#         # store the filtered dataset (only if relevant questions exist)
#         if all_questions:
#           print(f"Post has relevant follow-up questions in comments: {all_questions}")
#           row_filtered = [post.title, post.selftext, all_questions]
#           filtered_posts_data.append(row_filtered)

#     last_post = post.fullname

In [ ]:
# # determine if comment writer is a medical professional or not
# # okay if not a doctor as long as they have some medical training
# async def is_not_medical_professional(author, flair_text):
#   try:
#     flair_lower = flair_text.lower() if flair_text else ""
#     if "layperson" in flair_lower:
#       return True
#     if "not yet been verified" in flair_lower:
#       return True
#     if "unknown" in flair_lower:
#       return True
#   except Exception as e:
#     print(f"In is_not_medical_professional: Error fetching flair for {author}: {e}")
#     return True

#   return False

## Post Process

---



In [ ]:
# # go through each post with relevant comments and get a formatted list of the follow-up questions
# def extract_questions(comment_body):
#   # if starts at the beginning of text/period/exclamation mark/question mark and ends with a question mark
#   question_matches = re.findall(r'(?:^|\n|\.|!|\?)([^\n.!?]*\?)', comment_body.strip())
#   found = [question.strip() for question in question_matches]

#   # list of phrases that indicate a recommendation, with allowance for an adjective in between
#   exclude_patterns = [r"i\s+\w+\s+recommend", r"i\s+\w+\s+suggest"]
#   # list of common unhelpful one word questions generated by ChatGPT
#   exclude_one_word_patterns = [r"okay", r"ok", r"why", r"how", r"what", r"who", r"where", r"which", r"sure", r"really", r"explain", r"something", r"someone", r"anything", r"anyone"]
#   removed_questions = []
#   filtered_questions = []

#   for q in found:
#     # remove recommendations
#     if any(re.search(pattern, q.lower()) for pattern in exclude_patterns):
#       removed_questions.append(q)
#     # remove unhelpful one word questions like "why?"
#     elif len(q.split()) == 1 and any(re.search(pattern, q.lower()) for pattern in exclude_one_word_patterns):
#       removed_questions.append(q)
#     else:
#       filtered_questions.append(q)

#   # print if removed questions
#   if removed_questions:
#     print(f"removed_questions = {removed_questions}")

#   if not filtered_questions:
#     return None

#   return filtered_questions

## Write to Files

---



In [ ]:
# # writes data to csv file
# def save_to_csv(filename_orginal, filename_filtered):
#   global original_posts_data
#   global filtered_posts_data
#   print(f"Saving to csv.")

#   # FOR ORIGINAL
#   # if data is empty, write an empty CSV and return
#   if not original_posts_data:
#     with open(filename_orginal, mode="w", newline="", encoding="utf-8-sig") as file:
#       pass
#     return

#   # each comment has body and flair
#   max_comments = max(len(row) - 3 for row in original_posts_data) // 2
#   headers = ["Post Link", "Title", "Post Body"] + [col for i in range(max_comments) for col in (f"Comment {i+1}", f"Flair {i+1}")]

#   with open(filename_orginal, mode="w", newline="", encoding="utf-8-sig") as file:
#     writer = csv.writer(file)
#     writer.writerow(headers)
#     writer.writerows(original_posts_data)

#   # FOR FILTERED
#   # if data is empty, write an empty CSV and return
#   if not filtered_posts_data:
#     with open(filename_filtered, mode="w", newline="", encoding="utf-8-sig") as file:
#       pass
#     return

#   # each comment has body and flair
#   headers = ["Title", "Post Body", "Relevant Questions"]

#   with open(filename_filtered, mode="w", newline="", encoding="utf-8-sig") as file:
#     writer = csv.writer(file)
#     writer.writerow(headers)
#     writer.writerows(filtered_posts_data)

## Run Script

---



In [ ]:
# cvsname_original = "/content/drive/MyDrive/Colab Notebooks/askdocs_posts_questions.csv"
# cvsname_filtered = "/content/drive/MyDrive/Colab Notebooks/askdocs_filtered_posts_questions.csv"
# number_of_posts_to_find = 2000


# async def main():
#   # use pagination to bypass limit
#   while len(filtered_posts_data) < number_of_posts_to_find:
#     try:
#       # reddit API credentials
#       async with asyncpraw.Reddit(
#         client_id='m57EBw-RZefGEK2hn2Arag',
#         client_secret='5OO3OMGjzCiagd83GW7ioMWDNacw_g',
#         user_agent='cs72 askdocs scraper v1.0 by /u/Competitive-Year-702'
#       ) as reddit:

#         # run search
#         await fetch_posts(reddit, number_of_posts_to_find)
#         print(f"Saved {len(filtered_posts_data)} posts to data set.")

#         if len(original_posts_data) >= number_of_posts_to_find:
#           break  # stop when reaching the limit

#     except Exception as e:
#       print(f"Error fetching posts: {e}")
#       await asyncio.sleep(60)


# await main()

# save_to_csv(cvsname_original, cvsname_filtered)
# print(f"Saved {len(filtered_posts_data)} posts to final CSV.")